*By: Xujie Nuja Zhang & Yutung Tracy Huang*
***

In this file, we read in the Yelp review data, which we downloaded from [Kaggle](https://www.kaggle.com/yelp-dataset/yelp-dataset#yelp_academic_dataset_review.json). We then cleaned the data, broke the sentences into words, lemmetized the words, and got rid of the stop words, constructed bigram for each word, and trained the model using word2vec. We then output the model, which we will use in our web app (app.py).

Note: we referenced [this tutorial](https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial) for our construction of the model.

##### Data Gathering

In [19]:
data_directory = "yelp_academic_dataset_review.json"

In [33]:
import pandas as pd
import json
from tqdm import tqdm

In [56]:
# create & open a new file in write mode
with codecs.open("yelp_reviews.txt", 'w', encoding='utf_8') as review_txt_file:
    # open the existing review json file
    with codecs.open(data_directory, encoding='utf_8') as review_json_file:
        # loop through all reviews in the existing file and convert to dict
        for i in tqdm(range(1000000)):
            review_dict = json.loads(review_json_file.readline())
            review = review_dict['text']
#             print(review)
            if review == '\n': continue
            review_txt_file.write(review)


 85%|████████▍ | 847731/1000000 [00:26<00:04, 31348.38it/s]


100%|██████████| 1000000/1000000 [00:31<00:00, 31736.42it/s]

Text from 1,000,000 restaurant reviews
              written to the new txt file.


In [45]:
# read in the dataframe to file:
with open("yelp_reviews.txt",'r') as f: 
    reviews = f. readlines()
    
df = pd.DataFrame(reviews)
df.columns = ['review_text']
df = df[df.review_text != '\n']

##### Preprocess

In [57]:
# import the needed libraries
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [62]:
nlp = spacy.load('en', disable=['ner','parser']) # disabling Named Entity Recognition for speed

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [63]:
# remove non-alphabetic characters
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['review_text'])

In [64]:
t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 74.22 mins


In [65]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(2122309, 1)

In [66]:
df_clean.head()

,clean
0,total bill horrible service gs crook actually ...
1,adore travi hard rock new kelly cardena salon ...
2,travis greet perfectly green swoosh perfectly ...
3,travi start flat iron way flip wrist volume ma...
4,travi single time vegas feel beauuuutiful


##### Bigram

In [67]:
# this library lets us catch bigram phrases
from gensim.models.phrases import Phrases, Phraser

In [68]:
# prepare a list of words to be sent to Phrases() (list of lists)
sent = [row.split() for row in df_clean['clean']]

In [69]:
sent[:5]

[['total',
  'bill',
  'horrible',
  'service',
  'gs',
  'crook',
  'actually',
  'nerve',
  'charge',
  'pill',
  'check',
  'online',
  'pill',
  'cent',
  'avoid',
  'hospital',
  'ers',
  'cost'],
 ['adore',
  'travi',
  'hard',
  'rock',
  'new',
  'kelly',
  'cardena',
  'salon',
  'fan',
  'great',
  'blowout',
  'stranger',
  'chain',
  'offer',
  'service',
  'travi',
  'take',
  'flawless',
  'blowout',
  'new',
  'level'],
 ['travis',
  'greet',
  'perfectly',
  'green',
  'swoosh',
  'perfectly',
  'style',
  'black',
  'hair',
  'vegas',
  'worthy',
  'rockstar',
  'outfit',
  'come',
  'relaxing',
  'incredible',
  'shampoo',
  'head',
  'message',
  'cure',
  'bad',
  'migraine',
  'minute',
  'scented',
  'shampoo',
  'room',
  'travi',
  'freakishly',
  'strong',
  'finger',
  'good',
  'way',
  'use',
  'perfect',
  'pressure',
  'superb',
  'start',
  'glorious',
  'blowout',
  'people',
  'involve',
  'good',
  'round',
  'brush',
  'action',
  'hair',
  'see',
  '

In [70]:
# create relevant phrases
phrases = Phrases(sent, min_count=30, progress_per=10000)

INFO - 15:13:51: collecting all words and their counts
INFO - 15:13:51: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 15:13:51: PROGRESS: at sentence #10000, processed 219112 words and 164829 word types
INFO - 15:13:52: PROGRESS: at sentence #20000, processed 434500 words and 289228 word types
INFO - 15:13:52: PROGRESS: at sentence #30000, processed 653017 words and 400679 word types
INFO - 15:13:53: PROGRESS: at sentence #40000, processed 873122 words and 504415 word types
INFO - 15:13:53: PROGRESS: at sentence #50000, processed 1092386 words and 599745 word types
INFO - 15:13:54: PROGRESS: at sentence #60000, processed 1312884 words and 691270 word types
INFO - 15:13:54: PROGRESS: at sentence #70000, processed 1539975 words and 782638 word types
INFO - 15:13:55: PROGRESS: at sentence #80000, processed 1760185 words and 866668 word types
INFO - 15:13:55: PROGRESS: at sentence #90000, processed 1981069 words and 947509 word types
INFO - 15:13:56: PROGRESS: at sent

INFO - 15:14:29: PROGRESS: at sentence #860000, processed 18989206 words and 4741783 word types
INFO - 15:14:29: PROGRESS: at sentence #870000, processed 19209327 words and 4779309 word types
INFO - 15:14:30: PROGRESS: at sentence #880000, processed 19433015 words and 4815336 word types
INFO - 15:14:30: PROGRESS: at sentence #890000, processed 19655416 words and 4853174 word types
INFO - 15:14:31: PROGRESS: at sentence #900000, processed 19874718 words and 4890024 word types
INFO - 15:14:31: PROGRESS: at sentence #910000, processed 20095040 words and 4926383 word types
INFO - 15:14:32: PROGRESS: at sentence #920000, processed 20312580 words and 4962862 word types
INFO - 15:14:32: PROGRESS: at sentence #930000, processed 20534327 words and 4999996 word types
INFO - 15:14:32: PROGRESS: at sentence #940000, processed 20755035 words and 5037225 word types
INFO - 15:14:33: PROGRESS: at sentence #950000, processed 20975670 words and 5073374 word types
INFO - 15:14:33: PROGRESS: at sentence #

INFO - 15:15:07: PROGRESS: at sentence #1710000, processed 37869680 words and 7580060 word types
INFO - 15:15:07: PROGRESS: at sentence #1720000, processed 38091942 words and 7609257 word types
INFO - 15:15:08: PROGRESS: at sentence #1730000, processed 38315525 words and 7638890 word types
INFO - 15:15:08: PROGRESS: at sentence #1740000, processed 38531481 words and 7667054 word types
INFO - 15:15:09: PROGRESS: at sentence #1750000, processed 38752013 words and 7696084 word types
INFO - 15:15:09: PROGRESS: at sentence #1760000, processed 38970987 words and 7724420 word types
INFO - 15:15:09: PROGRESS: at sentence #1770000, processed 39190243 words and 7753248 word types
INFO - 15:15:10: PROGRESS: at sentence #1780000, processed 39406889 words and 7782024 word types
INFO - 15:15:10: PROGRESS: at sentence #1790000, processed 39627094 words and 7810504 word types
INFO - 15:15:11: PROGRESS: at sentence #1800000, processed 39847397 words and 7839363 word types
INFO - 15:15:11: PROGRESS: at 

In [71]:
bigram = Phraser(phrases)

INFO - 15:17:35: source_vocab length 8734834
INFO - 15:19:03: Phraser built with 11364 phrasegrams


In [72]:
sentences = bigram[sent]

In [73]:
# sanity check
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

213951

In [74]:
# show the top ten most frequent words
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['good',
 'place',
 'food',
 'time',
 'great',
 'come',
 'like',
 'order',
 'service',
 'go']

##### Train the model

The training of the model will be in 3 steps:
    1. set up the parameters of the model, parameter sentence missing: Word2Vec()
    2. initialze the model using: .build_vocab()
    3. .train()

In [75]:
import multiprocessing

from gensim.models import Word2Vec

In [76]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

In [77]:
# step 1
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [78]:
# step 2
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 15:22:46: collecting all words and their counts
INFO - 15:22:46: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 15:22:46: PROGRESS: at sentence #10000, processed 206067 words, keeping 17757 word types
INFO - 15:22:47: PROGRESS: at sentence #20000, processed 408411 words, keeping 25275 word types
INFO - 15:22:48: PROGRESS: at sentence #30000, processed 613995 words, keeping 30798 word types
INFO - 15:22:49: PROGRESS: at sentence #40000, processed 820973 words, keeping 35441 word types
INFO - 15:22:49: PROGRESS: at sentence #50000, processed 1027075 words, keeping 39179 word types
INFO - 15:22:50: PROGRESS: at sentence #60000, processed 1234357 words, keeping 42687 word types
INFO - 15:22:51: PROGRESS: at sentence #70000, processed 1447936 words, keeping 45879 word types
INFO - 15:22:52: PROGRESS: at sentence #80000, processed 1655075 words, keeping 48630 word types
INFO - 15:22:53: PROGRESS: at sentence #90000, processed 1862455 words, keeping 51004 word

INFO - 15:23:45: PROGRESS: at sentence #830000, processed 17232671 words, keeping 131880 word types
INFO - 15:23:46: PROGRESS: at sentence #840000, processed 17440519 words, keeping 132577 word types
INFO - 15:23:47: PROGRESS: at sentence #850000, processed 17647516 words, keeping 133238 word types
INFO - 15:23:47: PROGRESS: at sentence #860000, processed 17854584 words, keeping 133998 word types
INFO - 15:23:48: PROGRESS: at sentence #870000, processed 18061706 words, keeping 134746 word types
INFO - 15:23:49: PROGRESS: at sentence #880000, processed 18272028 words, keeping 135423 word types
INFO - 15:23:49: PROGRESS: at sentence #890000, processed 18481243 words, keeping 136193 word types
INFO - 15:23:50: PROGRESS: at sentence #900000, processed 18687406 words, keeping 136905 word types
INFO - 15:23:50: PROGRESS: at sentence #910000, processed 18894643 words, keeping 137647 word types
INFO - 15:23:51: PROGRESS: at sentence #920000, processed 19098923 words, keeping 138397 word types


INFO - 15:24:40: PROGRESS: at sentence #1650000, processed 34365037 words, keeping 187147 word types
INFO - 15:24:40: PROGRESS: at sentence #1660000, processed 34580520 words, keeping 187826 word types
INFO - 15:24:41: PROGRESS: at sentence #1670000, processed 34789306 words, keeping 188429 word types
INFO - 15:24:42: PROGRESS: at sentence #1680000, processed 34996651 words, keeping 189001 word types
INFO - 15:24:42: PROGRESS: at sentence #1690000, processed 35206871 words, keeping 189599 word types
INFO - 15:24:43: PROGRESS: at sentence #1700000, processed 35412310 words, keeping 190236 word types
INFO - 15:24:45: PROGRESS: at sentence #1710000, processed 35625039 words, keeping 190779 word types
INFO - 15:24:46: PROGRESS: at sentence #1720000, processed 35834066 words, keeping 191440 word types
INFO - 15:24:46: PROGRESS: at sentence #1730000, processed 36044870 words, keeping 192064 word types
INFO - 15:24:47: PROGRESS: at sentence #1740000, processed 36247950 words, keeping 192583 w

Time to build vocab: 2.56 mins


In [79]:
# step 3:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 15:27:20: training model with 3 workers on 39248 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2
INFO - 15:27:21: EPOCH 1 - PROGRESS: at 0.41% examples, 87361 words/s, in_qsize 0, out_qsize 0
INFO - 15:27:22: EPOCH 1 - PROGRESS: at 0.85% examples, 91504 words/s, in_qsize 0, out_qsize 0
INFO - 15:27:23: EPOCH 1 - PROGRESS: at 1.33% examples, 96234 words/s, in_qsize 0, out_qsize 0
INFO - 15:27:24: EPOCH 1 - PROGRESS: at 1.74% examples, 95116 words/s, in_qsize 0, out_qsize 0
INFO - 15:27:25: EPOCH 1 - PROGRESS: at 2.20% examples, 95975 words/s, in_qsize 0, out_qsize 0
INFO - 15:27:26: EPOCH 1 - PROGRESS: at 2.65% examples, 96386 words/s, in_qsize 0, out_qsize 0
INFO - 15:27:27: EPOCH 1 - PROGRESS: at 3.06% examples, 95284 words/s, in_qsize 0, out_qsize 0
INFO - 15:27:28: EPOCH 1 - PROGRESS: at 3.52% examples, 96134 words/s, in_qsize 0, out_qsize 0
INFO - 15:27:29: EPOCH 1 - PROGRESS: at 3.91% examples, 94097 words/s, in_qsize 0, out_qsize 0
INFO - 15:

INFO - 15:28:47: EPOCH 1 - PROGRESS: at 42.59% examples, 110552 words/s, in_qsize 0, out_qsize 0
INFO - 15:28:48: EPOCH 1 - PROGRESS: at 43.07% examples, 110465 words/s, in_qsize 0, out_qsize 0
INFO - 15:28:49: EPOCH 1 - PROGRESS: at 43.44% examples, 110092 words/s, in_qsize 0, out_qsize 0
INFO - 15:28:50: EPOCH 1 - PROGRESS: at 43.95% examples, 110151 words/s, in_qsize 0, out_qsize 0
INFO - 15:28:51: EPOCH 1 - PROGRESS: at 44.47% examples, 110223 words/s, in_qsize 0, out_qsize 0
INFO - 15:28:52: EPOCH 1 - PROGRESS: at 45.03% examples, 110390 words/s, in_qsize 0, out_qsize 0
INFO - 15:28:53: EPOCH 1 - PROGRESS: at 45.57% examples, 110499 words/s, in_qsize 0, out_qsize 0
INFO - 15:28:54: EPOCH 1 - PROGRESS: at 46.11% examples, 110602 words/s, in_qsize 1, out_qsize 0
INFO - 15:28:55: EPOCH 1 - PROGRESS: at 46.66% examples, 110702 words/s, in_qsize 0, out_qsize 0
INFO - 15:28:57: EPOCH 1 - PROGRESS: at 47.24% examples, 110803 words/s, in_qsize 0, out_qsize 0
INFO - 15:28:58: EPOCH 1 - PRO

INFO - 15:30:15: EPOCH 1 - PROGRESS: at 86.63% examples, 112971 words/s, in_qsize 0, out_qsize 0
INFO - 15:30:16: EPOCH 1 - PROGRESS: at 87.19% examples, 113025 words/s, in_qsize 0, out_qsize 0
INFO - 15:30:17: EPOCH 1 - PROGRESS: at 87.80% examples, 113156 words/s, in_qsize 0, out_qsize 0
INFO - 15:30:18: EPOCH 1 - PROGRESS: at 88.34% examples, 113195 words/s, in_qsize 0, out_qsize 0
INFO - 15:30:19: EPOCH 1 - PROGRESS: at 88.92% examples, 113233 words/s, in_qsize 0, out_qsize 0
INFO - 15:30:20: EPOCH 1 - PROGRESS: at 89.47% examples, 113292 words/s, in_qsize 0, out_qsize 0
INFO - 15:30:21: EPOCH 1 - PROGRESS: at 90.02% examples, 113331 words/s, in_qsize 0, out_qsize 0
INFO - 15:30:22: EPOCH 1 - PROGRESS: at 90.56% examples, 113377 words/s, in_qsize 0, out_qsize 0
INFO - 15:30:23: EPOCH 1 - PROGRESS: at 91.12% examples, 113454 words/s, in_qsize 0, out_qsize 0
INFO - 15:30:24: EPOCH 1 - PROGRESS: at 91.67% examples, 113514 words/s, in_qsize 0, out_qsize 0
INFO - 15:30:25: EPOCH 1 - PRO

INFO - 15:31:39: EPOCH 2 - PROGRESS: at 31.55% examples, 119993 words/s, in_qsize 0, out_qsize 0
INFO - 15:31:40: EPOCH 2 - PROGRESS: at 32.09% examples, 119933 words/s, in_qsize 0, out_qsize 0
INFO - 15:31:41: EPOCH 2 - PROGRESS: at 32.66% examples, 120018 words/s, in_qsize 0, out_qsize 0
INFO - 15:31:42: EPOCH 2 - PROGRESS: at 33.22% examples, 120115 words/s, in_qsize 0, out_qsize 0
INFO - 15:31:43: EPOCH 2 - PROGRESS: at 33.71% examples, 119969 words/s, in_qsize 0, out_qsize 0
INFO - 15:31:44: EPOCH 2 - PROGRESS: at 34.26% examples, 120011 words/s, in_qsize 0, out_qsize 0
INFO - 15:31:45: EPOCH 2 - PROGRESS: at 34.76% examples, 119834 words/s, in_qsize 0, out_qsize 0
INFO - 15:31:46: EPOCH 2 - PROGRESS: at 35.28% examples, 119791 words/s, in_qsize 0, out_qsize 0
INFO - 15:31:47: EPOCH 2 - PROGRESS: at 35.82% examples, 119808 words/s, in_qsize 0, out_qsize 0
INFO - 15:31:48: EPOCH 2 - PROGRESS: at 36.26% examples, 119554 words/s, in_qsize 0, out_qsize 0
INFO - 15:31:49: EPOCH 2 - PRO

INFO - 15:33:06: EPOCH 2 - PROGRESS: at 74.61% examples, 115972 words/s, in_qsize 1, out_qsize 0
INFO - 15:33:07: EPOCH 2 - PROGRESS: at 75.18% examples, 116047 words/s, in_qsize 0, out_qsize 0
INFO - 15:33:08: EPOCH 2 - PROGRESS: at 75.69% examples, 116049 words/s, in_qsize 0, out_qsize 0
INFO - 15:33:09: EPOCH 2 - PROGRESS: at 76.10% examples, 115880 words/s, in_qsize 0, out_qsize 0
INFO - 15:33:10: EPOCH 2 - PROGRESS: at 76.64% examples, 115925 words/s, in_qsize 0, out_qsize 0
INFO - 15:33:11: EPOCH 2 - PROGRESS: at 77.22% examples, 116024 words/s, in_qsize 0, out_qsize 0
INFO - 15:33:12: EPOCH 2 - PROGRESS: at 77.80% examples, 116116 words/s, in_qsize 0, out_qsize 0
INFO - 15:33:13: EPOCH 2 - PROGRESS: at 78.35% examples, 116160 words/s, in_qsize 0, out_qsize 0
INFO - 15:33:14: EPOCH 2 - PROGRESS: at 78.91% examples, 116234 words/s, in_qsize 0, out_qsize 0
INFO - 15:33:15: EPOCH 2 - PROGRESS: at 79.47% examples, 116303 words/s, in_qsize 0, out_qsize 0
INFO - 15:33:16: EPOCH 2 - PRO

...

INFO - 17:03:24: EPOCH 29 - PROGRESS: at 78.59% examples, 91894 words/s, in_qsize 0, out_qsize 0
INFO - 17:03:25: EPOCH 29 - PROGRESS: at 79.05% examples, 91922 words/s, in_qsize 1, out_qsize 0
INFO - 17:03:26: EPOCH 29 - PROGRESS: at 79.61% examples, 92097 words/s, in_qsize 0, out_qsize 0
INFO - 17:03:28: EPOCH 29 - PROGRESS: at 80.07% examples, 92122 words/s, in_qsize 0, out_qsize 0
INFO - 17:03:29: EPOCH 29 - PROGRESS: at 80.56% examples, 92218 words/s, in_qsize 0, out_qsize 0
INFO - 17:03:30: EPOCH 29 - PROGRESS: at 80.98% examples, 92241 words/s, in_qsize 0, out_qsize 0
INFO - 17:03:31: EPOCH 29 - PROGRESS: at 81.43% examples, 92291 words/s, in_qsize 0, out_qsize 0
INFO - 17:03:32: EPOCH 29 - PROGRESS: at 81.80% examples, 92230 words/s, in_qsize 0, out_qsize 0
INFO - 17:03:33: EPOCH 29 - PROGRESS: at 82.19% examples, 92152 words/s, in_qsize 0, out_qsize 0
INFO - 17:03:34: EPOCH 29 - PROGRESS: at 82.71% examples, 92263 words/s, in_qsize 0, out_qsize 0
INFO - 17:03:35: EPOCH 29 - PR

INFO - 17:04:49: EPOCH 30 - PROGRESS: at 14.36% examples, 104843 words/s, in_qsize 0, out_qsize 0
INFO - 17:04:50: EPOCH 30 - PROGRESS: at 14.81% examples, 104738 words/s, in_qsize 0, out_qsize 0
INFO - 17:04:51: EPOCH 30 - PROGRESS: at 15.27% examples, 104593 words/s, in_qsize 0, out_qsize 0
INFO - 17:04:52: EPOCH 30 - PROGRESS: at 15.77% examples, 104676 words/s, in_qsize 1, out_qsize 0
INFO - 17:04:53: EPOCH 30 - PROGRESS: at 16.23% examples, 104519 words/s, in_qsize 0, out_qsize 0
INFO - 17:04:54: EPOCH 30 - PROGRESS: at 16.70% examples, 104612 words/s, in_qsize 1, out_qsize 0
INFO - 17:04:55: EPOCH 30 - PROGRESS: at 17.20% examples, 104814 words/s, in_qsize 0, out_qsize 0
INFO - 17:04:56: EPOCH 30 - PROGRESS: at 17.70% examples, 104989 words/s, in_qsize 0, out_qsize 0
INFO - 17:04:57: EPOCH 30 - PROGRESS: at 18.18% examples, 105003 words/s, in_qsize 0, out_qsize 0
INFO - 17:04:58: EPOCH 30 - PROGRESS: at 18.59% examples, 104792 words/s, in_qsize 0, out_qsize 0
INFO - 17:04:59: EPO

INFO - 17:06:16: EPOCH 30 - PROGRESS: at 54.14% examples, 104410 words/s, in_qsize 0, out_qsize 0
INFO - 17:06:17: EPOCH 30 - PROGRESS: at 54.61% examples, 104352 words/s, in_qsize 0, out_qsize 0
INFO - 17:06:18: EPOCH 30 - PROGRESS: at 55.06% examples, 104303 words/s, in_qsize 0, out_qsize 0
INFO - 17:06:19: EPOCH 30 - PROGRESS: at 55.58% examples, 104413 words/s, in_qsize 0, out_qsize 0
INFO - 17:06:20: EPOCH 30 - PROGRESS: at 56.06% examples, 104430 words/s, in_qsize 0, out_qsize 0
INFO - 17:06:21: EPOCH 30 - PROGRESS: at 56.54% examples, 104510 words/s, in_qsize 0, out_qsize 0
INFO - 17:06:22: EPOCH 30 - PROGRESS: at 57.00% examples, 104503 words/s, in_qsize 0, out_qsize 0
INFO - 17:06:23: EPOCH 30 - PROGRESS: at 57.53% examples, 104638 words/s, in_qsize 0, out_qsize 0
INFO - 17:06:24: EPOCH 30 - PROGRESS: at 58.01% examples, 104700 words/s, in_qsize 0, out_qsize 0
INFO - 17:06:25: EPOCH 30 - PROGRESS: at 58.54% examples, 104843 words/s, in_qsize 0, out_qsize 0
INFO - 17:06:26: EPO

INFO - 17:07:42: EPOCH 30 - PROGRESS: at 93.44% examples, 104346 words/s, in_qsize 0, out_qsize 0
INFO - 17:07:43: EPOCH 30 - PROGRESS: at 93.96% examples, 104389 words/s, in_qsize 0, out_qsize 0
INFO - 17:07:44: EPOCH 30 - PROGRESS: at 94.41% examples, 104366 words/s, in_qsize 0, out_qsize 0
INFO - 17:07:45: EPOCH 30 - PROGRESS: at 94.85% examples, 104341 words/s, in_qsize 0, out_qsize 0
INFO - 17:07:46: EPOCH 30 - PROGRESS: at 95.30% examples, 104317 words/s, in_qsize 0, out_qsize 0
INFO - 17:07:47: EPOCH 30 - PROGRESS: at 95.66% examples, 104201 words/s, in_qsize 0, out_qsize 0
INFO - 17:07:48: EPOCH 30 - PROGRESS: at 96.14% examples, 104212 words/s, in_qsize 0, out_qsize 0
INFO - 17:07:49: EPOCH 30 - PROGRESS: at 96.54% examples, 104140 words/s, in_qsize 0, out_qsize 0
INFO - 17:07:50: EPOCH 30 - PROGRESS: at 97.07% examples, 104187 words/s, in_qsize 0, out_qsize 0
INFO - 17:07:51: EPOCH 30 - PROGRESS: at 97.49% examples, 104152 words/s, in_qsize 0, out_qsize 0
INFO - 17:07:52: EPO

Time to train the model: 100.63 mins


In [80]:
w2v_model.init_sims(replace=True)

INFO - 17:08:03: precomputing L2-norms of word weight vectors


In [3]:
# save the model to file
import pickle 

# save the model to disk
filename = 'word_algebra_model.sav'
pickle.dump(w2v_model, open(filename, 'wb'))

NameError: name 'w2v_model' is not defined

In [4]:
import pickle

##### Explore

In [5]:
# read in the model
model = pickle.load(open(filename, 'rb'))

In [12]:
model.wv.vocab.keys()

dict_keys(['total_bill', 'horrible', 'service', 'gs', 'crook', 'actually', 'nerve', 'charge', 'pill', 'check', 'online', 'cent', 'avoid', 'hospital', 'ers', 'cost', 'adore', 'travi', 'hard_rock', 'new', 'kelly', 'cardena', 'salon', 'fan', 'great', 'blowout', 'stranger', 'chain', 'offer', 'take', 'flawless', 'level', 'travis', 'greet', 'perfectly', 'green', 'style', 'black', 'hair', 'vegas', 'worthy', 'rockstar', 'outfit', 'come', 'relaxing', 'incredible', 'shampoo', 'head', 'message', 'cure', 'bad', 'migraine', 'minute', 'scented', 'room', 'freakishly', 'strong', 'finger', 'good', 'way', 'use', 'perfect', 'pressure', 'superb', 'start', 'glorious', 'people', 'involve', 'round', 'brush', 'action', 'see', 'team', 'stylist', 'clearly', 'get', 'extremely', 'evident', 'talk', 'help', 'genuine', 'corporate', 'requirement', 'fun', 'flat_iron', 'flip', 'wrist', 'volume', 'make', 'look', 'like', 'texas', 'girl', 'admirable', 'worth', 'note', 'fry', 'happen', 'skilled', 'end', 'terrific', 'thing'

In [14]:
words = {words for words in model.wv.vocab.keys() if len(words) > 3}

In [18]:
words

{'poppy_seed',
 'blta',
 'cancun',
 'price_reasonable',
 'rounder',
 'dipping_sauce',
 'siu_mai',
 'recreational',
 'jean_talon',
 'hibachi_grill',
 'meetup',
 'bradley',
 'lynch',
 'desert_landscape',
 'adriana',
 'iron_chef',
 'donn',
 'puff',
 'hairdryer',
 'feel_obligate',
 'packing',
 'interact',
 'pool_builder',
 'paneer_tikka',
 'toke',
 'spotlessly_clean',
 'skeptic',
 'peruse',
 'car_tow',
 'white_chocolate',
 'recooke',
 'taphouse',
 'coincidently',
 'ruse',
 'perez',
 'recette',
 'linq_hotel',
 'fanciness',
 'whitefish',
 'summation',
 'gordon_ramsay',
 'vote',
 'sushi_mon',
 'rivas',
 'boka',
 'uninvite',
 'stigma',
 'alligator',
 'royalton',
 'je_crois',
 'ingenious',
 'rhode_ranch',
 'camaro',
 'ageless',
 'dinning_area',
 'madam',
 'banging',
 'mundane',
 'lack_organization',
 'unexceptional',
 'daughter',
 "n'est",
 'button',
 'reputation',
 'selve',
 'tiabi',
 'sippy_cup',
 'accounting',
 'milliken',
 'albondigas',
 'upselling',
 'staple',
 'honest_ed',
 'solver',
 'gi

In [108]:
model.wv.most_similar(positive=["california", "ritz"],topn=3)

[('san_diego', 0.5028759241104126),
 ('los_angele', 0.502808690071106),
 ('cali', 0.49196651577949524)]

In [76]:
word_list = ['farmer_market','lobster','acai','santa_monica','breakfast','lunch','dinner','sandwich','guacamole','romantic','view','slot_machine','national_park','view','stunning','date','catalina','island','tempura_flake','sushi','wine','napa_valley','cheese','burger','grand_canyon','irvine','san_jose','orange_county','hollywood','taco','sequoia','scenic_drive','mexican','thai','chinese','spicy','hotpot','harry_potter','pac_man']

In [109]:
word_list = ['sequoia','orange_county','napa_valley','grand_canyon','irvine','san_jose','hollywood',"french_bistro","university_student","stern","renaissance","yummy","zucchini","skating","tonkatsu","papa_johns","grilled_fish","canape","continental_breakfast","domino","hibachi","tempura_roll","avocado_toast","shakshuka",'farmer_market','lobster','acai','santa_monica','breakfast','lunch','dinner','sandwich','guacamole','romantic','view','slot_machine','national_park','stunning','date','catalina','island','tempura_flake','sushi','wine','cheese','burger','taco','scenic_drive','mexican','thai','chinese','spicy','hotpot','harry_potter','pac_man']
for word in word_list:
    print(word)

french_bistro
university_student
stern
renaissance
yummy
zucchini
skating
tonkatsu
papa_johns
grilled_fish
canape
continental_breakfast
domino
hibachi
tempura_roll
avocado_toast
shakshuka
farmer_market
lobster
acai
santa_monica
breakfast
lunch
dinner
sandwich
guacamole
romantic
view
slot_machine
national_park
stunning
date
catalina
island
tempura_flake
sushi
wine
napa_valley
cheese
burger
grand_canyon
irvine
san_jose
orange_county
hollywood
taco
sequoia
scenic_drive
mexican
thai
chinese
spicy
hotpot
harry_potter
pac_man
